Link of deployed app: https://med7diagnosis.herokuapp.com/

In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import  Model, Sequential
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


In [3]:
IMAGE_SIZE = [224, 224]
train_path = 'datasets/chest_xray/train'
test_path = 'datasets/chest_xray/test'
val_path = 'datasets/chest_xray/val'

In [4]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in vgg.layers:
    layer.trainable = False

In [6]:
folders = glob(train_path + '/*')
folders

['datasets/chest_xray/train\\NORMAL', 'datasets/chest_xray/train\\PNEUMONIA']

In [7]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_data = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_data = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=32, class_mode='categorical')
val_data = test_datagen.flow_from_directory(val_path, target_size=(224, 224), batch_size=32, class_mode='categorical')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [11]:
class bestAccValAcc(tf.keras.callbacks.Callback):
    def __init__(self, val_data):
        super(bestAccValAcc, self).__init__()
        self.val_data = val_data
    def on_train_begin(self, logs=None):
        self.best_AVA = 0
    def on_epoch_end(self, epoch, logs=None):
        current_acc = logs.get('accuracy')
        loss, acc = self.model.evaluate(self.val_data, verbose=0)
        print('\nTrain Accuracy:', round(current_acc, 4))
        print('Val Accuracy:', round(acc, 4))
        if ((current_acc + acc)/2) > self.best_AVA:
            print('Found better Mode! Saving!')
            self.best_AVA = (current_acc + acc)/2
            self.model.save('pneumonia_model')

In [12]:
r = model.fit(train_data, validation_data=val_data, epochs=20, steps_per_epoch=len(train_data), validation_steps=len(val_data), callbacks=[bestAccValAcc(val_data = val_data)])

Epoch 1/20
163/163 [==============================] - ETA: 0s - loss: 0.3144 - accuracy: 0.8942
Train Accuracy: 0.8942
Val Accuracy: 0.8125
Found better Mode! Saving!
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: pneumonia_model\assets
163/163 [==============================] - 96s 591ms/step - loss: 0.3144 - accuracy: 0.8942 - val_loss: 0.3422 - val_accuracy: 0.8125
Epoch 2/20
163/163 [==============================] - ETA: 0s - loss: 0.1658 - accuracy: 0.9385
Train Accuracy: 0.9385
Val Accuracy: 0.8125
Found better Mode! Saving!
INFO:tensorflow:Assets written to: pneumonia_model\assets
163/163 [==============================] - 95s 581ms/step - loss: 0.1658 - accuracy: 0.9385 - val_loss: 0.5303 - val_accuracy: 0.8125
Epoch 3/20
163/163 [============================

In [13]:
from tensorflow.keras.preprocessing.image import load_img

model = tf.keras.models.load_model('pneumonia_model')

img = load_img('F:/dataScienceCompleteProjects/MedicalDiagnosis/datasets/chest_xray/test/NORMAL/IM-0065-0001.jpeg', target_size=(224,224))
x = image.img_to_array(img)
x/=255
x = np.expand_dims(x, axis=0)
print(np.argmax(model.predict(x), axis=1))

img = load_img('F:/dataScienceCompleteProjects/MedicalDiagnosis/datasets/chest_xray/test/PNEUMONIA/person40_virus_87.jpeg', target_size=(224,224))
x = image.img_to_array(img)
x/=255
x = np.expand_dims(x, axis=0)
print(np.argmax(model.predict(x), axis=1))

img = load_img('F:/dataScienceCompleteProjects/MedicalDiagnosis/datasets/chest_xray/test/NORMAL/NORMAL2-IM-0086-0001.jpeg', target_size=(224,224))
x = image.img_to_array(img)
x/=255
x = np.expand_dims(x, axis=0)
print(np.argmax(model.predict(x), axis=1))

img = load_img('F:/dataScienceCompleteProjects/MedicalDiagnosis/datasets/chest_xray/test/PNEUMONIA/person72_virus_133.jpeg', target_size=(224,224))
x = image.img_to_array(img)
x/=255
x = np.expand_dims(x, axis=0)
print(np.argmax(model.predict(x), axis=1))

[0]
[1]
[0]
[1]
